In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import os, sys
print(tf.__version__)
import pandas as pd
import math
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from seqeval.metrics import classification_report,accuracy_score,f1_score
from tqdm import tqdm,trange
sys.path.append('../citation_bio_trainer')
#from feature.SpacyFeaturizer import get_spacy_feats_from_text
from feature.Featurizer import Featurizer
from util.Utils import calulate_ser_jer, load_from_folder, pad_sequences, load_embedding_matrix, evaluate, log_mlflow_results
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


2.2.0


In [3]:
# %%time
# df_data = pd.read_csv("/nlp/ner_dataset.csv",sep=",",encoding="latin1").fillna(method='ffill')
# df_data.shape
# tag_list=df_data.Tag.unique()
# from sklearn.model_selection import train_test_split
# x_train,x_valid=train_test_split(df_data,test_size=0.20,shuffle=False)
# agg_func = lambda s: [ [w,t] for w,t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
# x_train_grouped = x_train.groupby("Sentence #").apply(agg_func)
# x_valid_grouped = x_valid.groupby("Sentence #").apply(agg_func)
# x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values][0:10000]
# x_valid_sentences = [[s[0] for s in sent] for sent in x_valid_grouped.values]
# x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values][0:10000]
# x_valid_tags = [[t[1] for t in tag] for tag in x_valid_grouped.values]

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
%%time
train_data_path = '/nlp/cs_mixed_20k/cs_mixed_20k_train/'
test_data_path  = '/nlp/cs_mixed_20k/cs_mixed_20k_test/'
eval_without_intra_newline_path  = '/nlp/eval_data_spacy_tokenized_extra_space_removed/'

sentences_train, sent_tags_train = load_from_folder(train_data_path)
sentences_test, sent_tags_test = load_from_folder(test_data_path)
sentences_eval1, sent_tags_eval1 = load_from_folder(eval_without_intra_newline_path)

CPU times: user 1min 25s, sys: 585 ms, total: 1min 26s
Wall time: 1min 26s


In [5]:
xtrain, xvalid, ytrain, yvalid = train_test_split(sentences_train, sent_tags_train, test_size=0.1, random_state=42)

In [6]:
%%time
x_train_sentences = [i.split(" ") for i in xtrain]
x_train_tags = [i.split(" ") for i in ytrain]
x_valid_sentences = [i.split(" ") for i in xvalid]
x_valid_tags = [i.split(" ") for i in yvalid]

CPU times: user 2.13 s, sys: 745 ms, total: 2.88 s
Wall time: 2.88 s


In [7]:
tag_list = {'B-CIT', 'I-CIT'}
num_labels = 2
num_labels
label_map = {'B-CIT': 1, 'I-CIT': 0}

In [8]:
BERT_MODEL="bert-base-multilingual-cased"
pad_token=0,
pad_token_segment_id=0,
sequence_a_segment_id=0,

In [9]:
from transformers import (
    TF2_WEIGHTS_NAME,
    BertConfig,
    BertTokenizer,
    TFBertForTokenClassification,
    create_optimizer)

In [10]:
MODEL_CLASSES = {"bert": (BertConfig, TFBertForTokenClassification, BertTokenizer)}

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
#label_map = {label: i for i, label in enumerate(tag_list)}

In [12]:
# num_labels = len(tag_list) + 1
# num_labels

In [13]:
label_map

{'B-CIT': 1, 'I-CIT': 0}

In [14]:
pad_token_label_id = 0

In [15]:
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

In [16]:
config = config_class.from_pretrained(BERT_MODEL,num_labels=num_labels)

In [17]:
#config

In [18]:
tokenizer = tokenizer_class.from_pretrained(BERT_MODEL,do_lower_case=False)

In [19]:
model = model_class.from_pretrained(
                BERT_MODEL,
                from_pt=bool(".bin" in BERT_MODEL),
                config=config)

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/tensorflow/python/training/tracking/data_structures.py:718: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from

In [20]:
model.layers[-1].activation = tf.keras.activations.softmax

In [34]:
from keras.preprocessing.sequence import pad_sequences
max_seq_length =512

def convert_to_input(sentences,tags):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]
  
  for x,y in tqdm(zip(sentences,tags),total=len(tags)):
  
    tokens = []
    label_ids = []

    for word, label in zip(x, y):
      word_tokens = tokenizer.tokenize(word)
      tokens.extend(word_tokens)
      # Use the real label id for the first token of the word, and padding ids for the remaining tokens
      label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]
      label_ids = label_ids[: (max_seq_length - special_tokens_count)]

    label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length, truncation=True)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

    label_id_list.append(label_ids)

  return input_id_list,token_type_id_list,attention_mask_list,label_id_list


In [35]:
input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)

100%|██████████| 15817/15817 [11:25<00:00, 23.08it/s]


In [36]:
input_ids_valid,token_ids_valid,attention_masks_valid,label_ids_valid=convert_to_input(x_valid_sentences,x_valid_tags)

100%|██████████| 1758/1758 [01:16<00:00, 23.04it/s]


In [37]:
input_ids_train = pad_sequences(input_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_train = pad_sequences(token_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_train = pad_sequences(attention_masks_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post").astype('int8')
label_ids_train = pad_sequences(label_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post").astype('int8')

In [38]:
np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train),

((15817, 512), (15817, 512), (15817, 512), (15817, 512))

In [39]:
input_ids_valid = pad_sequences(input_ids_valid,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_valid = pad_sequences(token_ids_valid,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_valid = pad_sequences(attention_masks_valid,maxlen=max_seq_length,dtype="long",truncating="post",padding="post").astype('int8')
label_ids_valid = pad_sequences(label_ids_valid,maxlen=max_seq_length,dtype="long",truncating="post",padding="post").astype('int8')

In [40]:
np.shape(input_ids_valid),np.shape(token_ids_valid),np.shape(attention_masks_valid),np.shape(label_ids_valid),

((1758, 512), (1758, 512), (1758, 512), (1758, 512))

In [53]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

train_ds = tf.data.Dataset.from_tensor_slices((input_ids_train,attention_masks_train,token_ids_train,label_ids_train)).map(example_to_features).shuffle(1000).batch(4).repeat(5)


valid_ds=tf.data.Dataset.from_tensor_slices((input_ids_valid,attention_masks_valid,token_ids_valid,label_ids_valid)).map(example_to_features).batch(1)

In [54]:
for x,y in valid_ds.take(5):
  print(x,y)

{'input_ids': <tf.Tensor: shape=(1, 512), dtype=int64, numpy=
array([[   101,  51529,    117,    160,    119,    160,    119,    113,
         10363,    114,    119,  10117,  39474,  16298,  15163,  10107,
         23068,  10114,  61027,  62256,  56304,  10376,    119,  10167,
           144,    119,    138,    119,  14160,  24207,  10920,    113,
         13114,    119,    114,    117,  12080,  27413,  10142,  14155,
         14108,    113,  11309,    119,  26089,    100,  29823,    114,
           119,  13744,  77155,  10237,    117,  70576,    131,  16198,
         10915,  70830,  10465,  63814,    119,  29981,  30266,  11169,
           117,    140,    119,    117,    111,    144,  97235,  10113,
           117,    155,    119,    113,  10202,    114,    119,  11982,
         91199,  20709,  13987,  11964,  49853,  20838,  10948,  16924,
         10111,  22899,  48101,    131,  10105,  13474,  10108,  48473,
           119,  14056,  12145,  14696,    117,  10413,    113,    127,
  

In [55]:
model.summary()

Model: "tf_bert_for_token_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 177,854,978
Trainable params: 177,854,978
Non-trainable params: 0
_________________________________________________________________


In [56]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [57]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [58]:
history = model.fit(train_ds, epochs=3, validation_data=valid_ds)

Epoch 1/3
13966/19775 [====================>.........] - ETA: 1:26:29 - loss: 0.3309 - accuracy: 0.9824

KeyboardInterrupt: 

In [ ]:
train_ds